In [13]:
import joblib
import sys
import os
alamat = "/".join(os.getcwd().split("\\")[:-2])
sys.path.append(alamat+'/praproses')
import ngrams as ng
from normalisasi import normalisasi as nrm
import token as tkn
import stemming as stm
import cleansing
import stemming
import seleksi_kata as sk
import pandas as pd
import isDigit as isd

In [14]:
tdidf = joblib.load("model/tdidf.w")
naive_bayes = joblib.load("model/naive_bayes.m")

In [15]:
fitur = tdidf.get_feature_names()
kelas  = naive_bayes.classes_
kelas
postif = naive_bayes.feature_log_prob_ [1]
negatif = naive_bayes.feature_log_prob_ [0]
dict_ = {
    #'features':fitur,
    'positive':postif,
    'negative':negatif,
}
df_bobot = pd.DataFrame.from_dict(dict_)
df_bobot.to_excel("hasil_training.xlsx")
df_bobot

,positive,negative
0,-7.733685,-9.460691
1,-8.128291,-7.132576
2,-8.232101,-9.460691
3,-8.018842,-9.460691
4,-5.897480,-7.291395
...,...,...
2371,-9.442547,-7.846073
2372,-8.226094,-9.460691
2373,-8.291441,-9.460691
2374,-8.028081,-9.460691


In [16]:
def praproses(twt, n=1):
    twt = sk.seleksi(twt)
    twt = cleansing.cleaning(twt)
    twt = isd.remove_digit(twt)
    twt = nrm.slangword(twt)
    twt = stm.stemming(twt)
    twt = stm.StopWordRemover(twt)
    twt = ng.ngramku(twt, n=n)['string']
    return twt

In [17]:
praproses("RT @hpsetyawan: @afiadi setuju bgt", n=1)

'tuju banget'

In [18]:
kalimat = "RT @hpsetyawan: setuju bgt"
# print(praproses(kalimat))
naive_bayes.predict(tdidf.transform([praproses(kalimat)]))[0]
#memprediksi kalimat di preproses, kemudian di transform menjadi vector,diprediksi menggunakan NB

'positive'

In [19]:
# pwd

# Prediksi data uji [Merdeka Belajar]

In [20]:
df = pd.read_excel("../../data_uji/PPDB ZONASI.xlsx")
twet_uji = df.Twit.tolist()
label_asli = df.label.tolist()
twet_uji_bersih = list()

for ix, t in enumerate(twet_uji):
    if type(t) is float: #koding tambahan
        print(ix)       #koding tambahan
        t=" - "         #koding tambahan
        
    twet_uji_bersih.append(praproses(t))
    #print(ix)
prediksi = naive_bayes.predict(tdidf.transform(twet_uji_bersih))

from sklearn.metrics import classification_report
y_true = label_asli
y_pred = prediksi
target_names = ['negative', 'positive',]

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred,labels=["positive", "negative"]))
print(classification_report(y_true, y_pred, target_names=target_names))

[[45  5]
 [ 3 47]]
              precision    recall  f1-score   support

    negative       0.90      0.94      0.92        50
    positive       0.94      0.90      0.92        50

    accuracy                           0.92       100
   macro avg       0.92      0.92      0.92       100
weighted avg       0.92      0.92      0.92       100



In [21]:
ix = 0
for twet, asli, pred in zip(twet_uji, label_asli,prediksi):
    if pred!= asli:
        print(ix)
        print("Prediksi:",pred)
        print("asli    :",asli)
        print("    ",twet)
        print("="*100)
    ix+=1

1
Prediksi: positive
asli    : negative
     RT @bagas: Anuuu... pemahamannya kurang eh.. #merdekabelajar
9
Prediksi: positive
asli    : negative
     RT @gw_eilo: Malam Pak @nadiem saya izin tidur ya Pak, ngantuk, bosan sama kebijakan. #merdekabelajar
13
Prediksi: negative
asli    : positive
     RT @CNNIDdaily: Tapi masalah nomor zonasi jadi masalah urutan tuduh-menuduh saya kan juga banyak dituduh pak.
26
Prediksi: positive
asli    : negative
     RT @unknownzoel_: Saya lihat pak @nadiem percaya oleh guru mampu memajukan memberikan yang terbaik
39
Prediksi: negative
asli    : positive
     RT @trenzsolo: "Pak nadiem ini tidak percaya pada guru kita !!" Kick telak atas pernyataan 
44
Prediksi: negative
asli    : positive
     RT @bseptian15: @TeddyGusnaidi Sekarang berani tidak pak @nadiem menolak skripsi dan tesis masuk 
55
Prediksi: negative
asli    : positive
     RT @BagusStefanus: Ketika @nadiem tukang tipu dan korup tapi tetap dijadikan menteri
97
Prediksi: negative
asli    : p

In [11]:
import numpy as np

In [12]:
np.log(0.02)

-3.912023005428146